In [ ]:
import tensorflow as tf
import numpy as np

In [2]:
import numpy as np
import torch

shape = np.stack([
    35,
    100
], axis=0)

print(shape)

[ 35 100]


In [ ]:
from read_dataset import generator
import os

In [ ]:
tr_path = os.getcwd() + "\\..\\data\\sample_data\\train\\"

In [ ]:
tr_path

In [ ]:
from datanetAPI import DatanetAPI
#https://github.com/knowledgedefinednetworking/datanetAPI/tree/challenge2020
import networkx as nx

tool = DatanetAPI(tr_path)
it = iter(tool)
for sample in it:
    break

In [ ]:
network = sample.topology_object
nx.draw_networkx(network)

In [ ]:
network.edges, len(network.edges)

In [ ]:
i = 0
for edge in network.edges:
    print(network.edges[edge[0],edge[1],edge[2]])
    i+=1
    if i == 10:
        break

In [ ]:
network.nodes[0]

In [ ]:
network[0][1]

In [ ]:
# routing matrix tells you the path that needs to be taken for every source-dest node pair

print(sample.get_routing_matrix()[0])
print(sample.get_routing_matrix()[1])
print(sample.get_routing_matrix().shape)


In [ ]:
routing = sample.get_routing_matrix()
paths = routing[~np.eye(routing.shape[0], dtype=bool)].reshape(routing.shape[0], -1)
print(routing.shape, paths.shape)
paths = paths.flatten()
print(paths.shape, paths[:2],paths[98])

In [ ]:
cap_mat = np.full((network.number_of_nodes(), network.number_of_nodes()), fill_value=None)
for node in range(network.number_of_nodes()):
    for adj in network[node]:
        cap_mat[node, adj] = network[node][adj][0]['bandwidth']

print(cap_mat)
# take the indexes of where there is capacity and turn into a list
links = np.where(np.ravel(cap_mat) != None)[0].tolist()
link_capacities = (np.ravel(cap_mat)[links]).tolist()
len(links),len(link_capacities),links[:10],link_capacities[:10]

In [ ]:

nodes = len(routing)
ids = list(range(len(links)))

links_id = dict(zip(links, ids))
path_ids = []
for path in paths:                    # 182 lists of node sequences (src-> int -> .... -> dest)
    new_path = [] 
    for i in range(0, len(path) - 1): # number of nodes in the specific src,dest path... minus 1
        src = path[i]                 # break path into series of single hops (src,int1) (int1,int2) (int2,dst)
        dst = path[i + 1]
        new_path.append(links_id[src * nodes + dst])     # grabs the id of the link between each single hop (src,dest)
    path_ids.append(new_path)                            # aggregates a list of links used in all paths, links can be
                                                         # used more than once

In [ ]:
# convert paths from path described by nodes and turn into paths defined by links
len(path_ids),path_ids[98],paths[98]

In [ ]:
path_ids[:4]

In [ ]:
link_indices = []
path_indices = []
sequ_indices = []
segment = 0                                         
for p in path_ids:
    link_indices += p
    path_indices += len(p) * [segment]
    sequ_indices += list(range(len(p)))
    segment += 1

In [ ]:
# take path list and flatten into single individual list
# list of all links used in all paths, list of which path each link belongs to, which number each link is as part of its path
link_indices[:5],path_indices[:5],sequ_indices[:5]

In [ ]:
len(link_indices),len(path_indices),len(sequ_indices)

In [ ]:
traffic = sample.get_traffic_matrix()
traffic = traffic[~np.eye(traffic.shape[0], dtype=bool)].reshape(traffic.shape[0], -1)
traffic.shape
traffic[0,0]

In [ ]:
result = sample.get_performance_matrix()
result = result[~np.eye(result.shape[0], dtype=bool)].reshape(result.shape[0], -1)
print(result.shape)

sample.get_performance_matrix()[0,1]

In [ ]:
avg_bw = []
pkts_gen = []
delay = []
########## critical information here #############
for i in range(result.shape[0]):
    for j in range(result.shape[1]):
        flow = traffic[i, j]['Flows'][0]
        avg_bw.append(flow['AvgBw'])
        pkts_gen.append(flow['PktsGen'])
        d = result[i, j]['AggInfo']['AvgDelay']
        delay.append(d)

n_paths = len(path_ids)
n_links = max(max(path_ids)) + 1

In [ ]:
n_paths = len(path_ids)
n_links = max(max(path_ids)) + 1
n_links,n_paths

In [ ]:
len(avg_bw)

In [ ]:
for batch, delay in generator(tr_path):
    x = batch
    y = delay
    break

In [ ]:
# in this sample the capacity matrix is 14 x 14 and they remove the entries along the diagonal and then they flatten it

In [ ]:
x.keys()

In [ ]:
type(x['bandwith']),len(x['bandwith']),x['bandwith'][:10]

In [ ]:
type(x['packets']),len(x['packets']),x['packets'][:10]

In [ ]:
type(x['link_capacity']),len(x['link_capacity']),x['link_capacity'][:10]

In [ ]:
type(x['links']),len(x['links']),x['links'][:10]

In [ ]:
x['n_paths'],len(x['paths'])

In [ ]:
len(y),y[:10]